In [1]:
from neuron import h
import numpy as np
from neuron.units import mV,ms,um
import matplotlib.pyplot as plt
import os
import pandas as pd
import stim
from savedata import saveparams, save_es, savedata, saveplot
import all_voltages
import record_voltages_gpt

# import load_files
h.load_file("stdrun.hoc")
h.load_file("interpCoordinates.hoc")
h.load_file("setPointers.hoc")
h.load_file("calcVe_noGUI.hoc")
h.load_file("cellChooser.hoc")
h.load_file("setParams.hoc")
h.load_file("editMorphology.hoc")
h.load_file("plot_max.hoc")
h.load_file("field.hoc")



	1 


1.0

In [2]:
cell_id=1
var="cfreq"
data_dir = "C:\\Users\\Pc\\Documents\\Charite\\NEURON\\Data_processing"
bot_dir=os.path.join(data_dir,str(cell_id),var)

In [3]:
h.setParamsAdultHuman()
h.myelinate_ax=1
h.cell_chooser(cell_id)
cell_name = h.cell_names.o(cell_id-1).s  # `.s` converts HOC String to Python string
cell=h.cell

h.theta = 180
h.phi = 0
h.stim_mode=2
h.getes()

	1 
	1 
	1 
	1 
	1 
	1 
	1 
*---------*
Loaded Blue Brain cell: L1_NGC-DA_bNAC219_1
numSect was 293
Created List of SectionRefs for 293 sections
Assigned section types to each section in type_xtra
Assigned branch orders using traverse_tree (assign_order2) in order_xtra
Got main axon (method 2)
Myelinating axon: Replacing 234 Axonal sections w/ 241 Myelin, 241 Node, 102 Unmyelin sections
No myelin before 1st bifurcation
Scaled diameter of myelin sections using variable g-ratio
numSect was 643
Created List of SectionRefs for 643 sections
Assigned section types to each section in type_xtra
Assigned branch orders using traverse_tree (assign_order2) in order_xtra
Inserted xtra and extracellular in all 761 compartments
Got main axon (method 2)
Cell Loaded
Calculated potentials for theta = 180 deg, phi = 0 deg


0.0

In [5]:
from maxshift_plot import plot_maxshift
fig=plot_maxshift(bot_dir,filename="max_v",cell=cell,max_shift=None)
fig.show()

761
Loaded 761 max_shift values.
Total segments in cell: 761
